# 제주 한달살기 숙소 추천을 위한 분석 - 전처리


### 미스터멘션 - https://www.mrmention.co.kr/
### 한달살기 전문사이트
- 이유 : 한달살기를 위한 시설이 준비된 숙소
- case 155건

In [1]:
import pandas as pd
import numpy as np

import re

In [10]:
df_base = pd.read_csv('./data/mmention_20240109.csv')
df_base.head(3)

,title,condition,rating,reviewNum,location
0,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',6박,4.7,18,한림읍
1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,6박,4.7,24,애월읍
2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,6박,4.8,245,노형동


In [11]:
# 이름이 같은 숙소가 3개 - 같은 숙소의 다른 호실로 판단됨
df_base[df_base['title'].duplicated() == 1] 

,title,condition,rating,reviewNum,location
105,[제주/구좌] 덕천씨엘빌,29박,4.7,9,구좌읍


In [12]:
# 숙소가 2개로 등록되어서 하나로 바꾼다.
df_base[df_base['title'] == '[제주/구좌] 덕천씨엘빌'] 

,title,condition,rating,reviewNum,location
59,[제주/구좌] 덕천씨엘빌,29박,4.9,11,구좌읍
105,[제주/구좌] 덕천씨엘빌,29박,4.7,9,구좌읍


In [13]:
# 두개의 정보를 하나로 합치기
new_rating = round(((4.9 * 11) + (4.7 * 9)) / 20, 1)
temp = pd.DataFrame([['[제주/구좌] 덕천씨엘빌', '29박', new_rating, 20, '구좌읍']])
temp.columns = ['title', 'condition', 'rating', 'reviewNum', 'location']
temp

,title,condition,rating,reviewNum,location
0,[제주/구좌] 덕천씨엘빌,29박,4.8,20,구좌읍


In [14]:
temp2 = df_base[df_base['title'] != '[제주/구좌] 덕천씨엘빌'] 
df_base = pd.concat([temp, temp2])
df_base.reset_index(drop = True, inplace= True)
df_base

,title,condition,rating,reviewNum,location
0,[제주/구좌] 덕천씨엘빌,29박,4.8,20,구좌읍
1,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',6박,4.7,18,한림읍
2,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,6박,4.7,24,애월읍
3,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,6박,4.8,245,노형동
4,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,14박,4.5,82,안덕면
...,...,...,...,...,...
149,[제주/애월] 오션뷰 풀옵션 타운하우스 / 야외 족욕까지 매력적인 트윈픽스,29박,4.8,4,애월읍
150,[제주/애월] 제주 솔한옥,6박,4.9,103,애월읍
151,"[제주/한림] 반려견과 함께 비양도를 내 품안에, 해피데이",6박,4.7,56,한림읍
152,[제주시] 신축풀옵숙소 한달살기 '라온스테이',6박,4.5,10,공설로5길


In [15]:
df_base = df_base.reset_index()
df_base.rename(columns= {'index' : 'ID'}, inplace=True) 
df_base

,ID,title,condition,rating,reviewNum,location
0,0,[제주/구좌] 덕천씨엘빌,29박,4.8,20,구좌읍
1,1,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',6박,4.7,18,한림읍
2,2,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,6박,4.7,24,애월읍
3,3,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,6박,4.8,245,노형동
4,4,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,14박,4.5,82,안덕면
...,...,...,...,...,...,...
149,149,[제주/애월] 오션뷰 풀옵션 타운하우스 / 야외 족욕까지 매력적인 트윈픽스,29박,4.8,4,애월읍
150,150,[제주/애월] 제주 솔한옥,6박,4.9,103,애월읍
151,151,"[제주/한림] 반려견과 함께 비양도를 내 품안에, 해피데이",6박,4.7,56,한림읍
152,152,[제주시] 신축풀옵숙소 한달살기 '라온스테이',6박,4.5,10,공설로5길


In [16]:
df_base.to_csv('./data/mmention_base_ver1.csv', index=False)

디테일 df 정리

In [17]:
df_detail =pd.read_csv('./data/mmention_review_20240109.csv')
df_detail.head(3)

,name,review,review_date,star,ammen
0,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',한달살기 하면서 날씨도 좋고 모든게 좋았습니다~~단지내에 계시던 주민모두 너무 친절...,2022년 12월,20.0,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
1,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',"일년을 고민 끝에 떠난 제주도,,,\n자는 곳이 제일 중해서 고심 끝에 결정했습니다...",2022년 12월,20.0,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
2,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',퇴사하고 퇴직금 털어서 부모님 모시고 다녀왔습니다.\n독채는 신경쓸게 많을 것 같아...,2022년 12월,20.0,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...


In [18]:
df_detail['review'] = df_detail['review'].apply(lambda x: x.replace("\n", ' ').replace("\r", ''))
df_detail['review']

0       한달살기 하면서 날씨도 좋고 모든게 좋았습니다~~단지내에 계시던 주민모두 너무 친절...
1       일년을 고민 끝에 떠난 제주도,,, 자는 곳이 제일 중해서 고심 끝에 결정했습니다,...
2       퇴사하고 퇴직금 털어서 부모님 모시고 다녀왔습니다. 독채는 신경쓸게 많을 것 같아서...
3       9/1부터 2주간 제주에서 지내기 위해서 정한 스테이입니다. 큰도로에서 좀 들어와 ...
4       4층이지만 엘리베이터가 있어서 전혀불편함 없이 생활했어요. 스테이상태가 너무좋아서 ...
                              ...                        
3675    사실과다른내용이면  고지없이 삭제한다니  마음에들지 않으면 지우겠다는거네요 알아서 ...
3676    시외버스터미널과 가까워서 제주 어디로든 대중교통을 이용해 이동하기 편해요. 방도 두...
3677    일반 원투룸형 숙소로 공항에서 가깝고 주차도 편리하다.. 주택가라 조용해서 좋았는데...
3678    공항에서 택시비 5000원도 안나와요  도착첫날이나 마지막날 이용하기 좋아요 3층인...
3679    공항에서 그리 멀지 않아 접근성은 좋다..집기류들도 잘 갖춰서 괜찮으나 결정적으로 ...
Name: review, Length: 3680, dtype: object

In [19]:
df_detail['ammen'] = df_detail['ammen'].apply(lambda x: x.replace("\n", ' ').replace("\r", ''))
df_detail['ammen']

0       생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
1       생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
2       생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
3       생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
4       생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
                              ...                        
3675    생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 ...
3676    생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 ...
3677    생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 ...
3678    생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 ...
3679    생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 ...
Name: ammen, Length: 3680, dtype: object

In [20]:
# 수집할때 20을 나눠야하는데 5만 나눠서 추가로 4를 나눠주기
df_detail['star'] = df_detail['star'].apply(lambda x: int(x/4))
df_detail.head(3)

,name,review,review_date,star,ammen
0,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',한달살기 하면서 날씨도 좋고 모든게 좋았습니다~~단지내에 계시던 주민모두 너무 친절...,2022년 12월,5,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
1,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',"일년을 고민 끝에 떠난 제주도,,, 자는 곳이 제일 중해서 고심 끝에 결정했습니다,...",2022년 12월,5,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
2,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',퇴사하고 퇴직금 털어서 부모님 모시고 다녀왔습니다. 독채는 신경쓸게 많을 것 같아서...,2022년 12월,5,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...


In [ ]:
# 두 데이터 프레임 합치기

In [21]:
df_detail2 = df_detail.drop_duplicates(ignore_index = True, subset='name')
df_detail2 = df_detail2[['name', 'ammen']]
df_detail2

,name,ammen
0,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,생활시설 주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면...
2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
3,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
4,[제주시] 제주공항 근처 나만 알고 싶은 포근한 대가족 펜션 / 키즈 펜션,생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 건조기 도어락 세...
...,...,...
118,[제주/한림] 해넘이달 렌탈하우스,생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 수건 헤어 드라이...
119,[제주/애월] 오션뷰 풀옵션 타운하우스 / 야외 족욕까지 매력적인 트윈픽스,생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 건조기 도어락 세...
120,[제주/애월] 제주 솔한옥,생활시설 주차장 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 헤어 드라이기 ...
121,"[제주/한림] 반려견과 함께 비양도를 내 품안에, 해피데이",생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세면도구 수건 헤어 드라이기 세탁...


In [22]:
df = pd.merge(df_detail2, df_base, how='left', left_on='name', right_on='title')
df

,name,ammen,ID,title,condition,rating,reviewNum,location
0,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...,1.0,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',6박,4.7,18.0,한림읍
1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,생활시설 주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면...,2.0,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,6박,4.7,24.0,애월읍
2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...,3.0,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,6박,4.8,245.0,노형동
3,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...,4.0,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,14박,4.5,82.0,안덕면
4,[제주시] 제주공항 근처 나만 알고 싶은 포근한 대가족 펜션 / 키즈 펜션,생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 건조기 도어락 세...,5.0,[제주시] 제주공항 근처 나만 알고 싶은 포근한 대가족 펜션 / 키즈 펜션,6박,5.0,75.0,신비마을1길
...,...,...,...,...,...,...,...,...
118,[제주/한림] 해넘이달 렌탈하우스,생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 수건 헤어 드라이...,148.0,[제주/한림] 해넘이달 렌탈하우스,6박,5.0,11.0,한림읍
119,[제주/애월] 오션뷰 풀옵션 타운하우스 / 야외 족욕까지 매력적인 트윈픽스,생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 건조기 도어락 세...,149.0,[제주/애월] 오션뷰 풀옵션 타운하우스 / 야외 족욕까지 매력적인 트윈픽스,29박,4.8,4.0,애월읍
120,[제주/애월] 제주 솔한옥,생활시설 주차장 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 헤어 드라이기 ...,150.0,[제주/애월] 제주 솔한옥,6박,4.9,103.0,애월읍
121,"[제주/한림] 반려견과 함께 비양도를 내 품안에, 해피데이",생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세면도구 수건 헤어 드라이기 세탁...,151.0,"[제주/한림] 반려견과 함께 비양도를 내 품안에, 해피데이",6박,4.7,56.0,한림읍


In [23]:
df.to_csv('./data/mmention_base_ver1.csv', index = False)